## Using the ONNX model file for predictions
### In this Notebook we take the model trained, saved in ONNX format and we do some predictions using a sampled test dataset

In [ ]:
import time
import os
import random as rn

import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

import tensorflow as tf

# conda env: mlcpuv1
import keras2onnx
import onnxruntime as rt

In [3]:
# check TF version (> 2.3)
print('TF version', tf.__version__)
print('ONNX runtime version', rt.__version__)
print('keras2onnx version', keras2onnx.__version__)

TF version 2.3.1
ONNX runtime version 1.4.0
keras2onnx version 1.7.0


### prepare the test dataset

In [5]:
# prepare the dataset for test

# we take the dataset from Sklearn
data = load_breast_cancer(as_frame=True)

# I prefer working with Dataframe
orig_df = data.frame

# we must rename columns, to remove spaces in names
# otherwise we get problems with ONNX

# substitute all spaces with _
dict_columns = {}

for col in orig_df.columns:
    dict_columns[col] = col.replace(" ", "_")

orig_df = orig_df.rename(columns=dict_columns)

# Split the dataset in train, valid, test

N_TOTAL = orig_df.shape[0]
FRAC_TRAIN = 0.7
FRAC_VALID = 0.15

N_TRAIN = int(N_TOTAL * FRAC_TRAIN)
N_VALID = int(N_TOTAL * FRAC_VALID)
N_TEST = N_TOTAL - N_TRAIN - N_VALID

print('Numbers of samples for (total, train, valid, test):', N_TOTAL, N_TRAIN, N_VALID, N_TEST)

# shuffle the data
orig_df = orig_df.sample(frac=1.)

df_train = orig_df.iloc[:N_TRAIN]
df_valid = orig_df.iloc[N_TRAIN:N_TRAIN+N_VALID]
df_test = orig_df.iloc[N_TRAIN+N_VALID:]

Numbers of samples for (total, train, valid, test): 569 398 85 86


In [6]:
# to convert in TF dataset
#adapted from https://www.tensorflow.org/tutorials/structured_data/feature_columns
def df_to_dataset(df, predictor,  batch_size=32, shuffle=True):
    df = df.copy()
    labels = df.pop(predictor)
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    
    if shuffle:
        # don't shuffle test
        ds = ds.shuffle(buffer_size=len(df))
        
    ds = ds.batch(batch_size)
    return ds

In [8]:
# we take only the test dataset
ds_test = df_to_dataset(df_test, 'target', batch_size=16, shuffle=False)

### load the ONNX model

In [9]:
# load the ONNX model
ONNX_MODEL_FILE = 'modelbc-artifact/modelbc.onnx'

# first, a function to load the model and create an ONNX session
def create_session(onnx_file_name, print_info=False):
    sess = rt.InferenceSession(onnx_file_name)
    
    print('Loading OK')
    
    if print_info:
        print("ONNX model expects ", len(sess.get_inputs()), 'features:')
        # prints names of features
        for n, input in enumerate(sess.get_inputs()):
            print(input.name)
        
    return sess

# build the input as expected from ONNX runtime
def build_input_feed(f_batch):
    # input: the features_batch as extracted from TF dataset
    # devo costruire il dict come se lo aspetta onnx
    
    input_dict = {}
    
    # ogni feature è un singolo valore
    for col in f_batch.keys():
        # get the numpy array of values
        values = f_batch[col].numpy()
        n_rows = len(values)
        input_dict[col] = values.reshape((n_rows, 1))
    
    # input_feed is the dictionary input to ONNX model
    # for every feature a column vector (n_rows, 1)
    return input_dict

def onnx_predict(f_batch, sess=create_session(ONNX_MODEL_FILE)):
    # transform the input
    input_feed = build_input_feed(f_batch)
    
    # run inference
    pred_onnx = sess.run(None, input_feed)
    
    # build output
    output = {}
    output['probs'] = pred_onnx[0]
    
    return output

Loading OK


### do the test

In [18]:
# this way I take only the feature batch out of a dataset batch

# do the test on the entire test dataset
for i, f_batch in enumerate(iter(ds_test)):
    print('Batch n.', i+1)
    
    # needed
    f_batch = f_batch[0]
    
    tStart = time.time()
    
    onnx_probs = onnx_predict(f_batch)
    tEla = time.time() - tStart
    print('Predictions:', onnx_probs['probs'].ravel())
    
    print('Time (sec.) for batch prediction:', round(tEla, 3))
    print('')
    
print('')
print('ONNX test OK.')

Batch n. 1
Predictions: [9.9996328e-01 2.1381915e-02 9.9985421e-01 3.5762787e-06 9.3315327e-01
 9.9879181e-01 1.0000000e+00 9.9731231e-01 1.7906627e-01 3.2621622e-04
 9.9987954e-01 9.9990857e-01 1.0621250e-03 2.5789142e-03 9.9996686e-01
 9.9999344e-01]
Time (sec.) for batch prediction: 0.001

Batch n. 2
Predictions: [9.9691486e-01 9.9984485e-01 9.9589956e-01 1.9285083e-04 7.7962875e-05
 9.9998009e-01 0.0000000e+00 9.9987197e-01 0.0000000e+00 0.0000000e+00
 9.9915004e-01 9.9999642e-01 9.9996775e-01 2.3880601e-04 9.9728799e-01
 9.9998200e-01]
Time (sec.) for batch prediction: 0.003

Batch n. 3
Predictions: [2.9720610e-01 0.0000000e+00 9.9977744e-01 9.9484450e-01 6.7836046e-04
 9.9987328e-01 4.1628748e-02 9.9999833e-01 2.4653971e-03 9.7196364e-01
 9.9999535e-01 9.9933505e-01 9.9995828e-01 9.9156666e-01 9.9999392e-01
 9.9244332e-01]
Time (sec.) for batch prediction: 0.001

Batch n. 4
Predictions: [3.0377507e-04 0.0000000e+00 1.0000000e+00 9.9999702e-01 4.9226868e-01
 1.0584801e-02 9.997637

#### Ok, to do a prediction on 16 samples it takes around 2 msec.